In [65]:
import torch
from torchtext import vocab

In [172]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

tk = Tokenizer(BPE(unk_token="[UNK]"))
# tk.pre_tokenizer = Whitespace()
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tk.enable_padding(pad_id=3)

files = ["data/shahname_src.txt", "data/shahname_tgt.txt"]
tk.train(files, trainer)

text = " چو ترکان بدیدند کارجاسپ رفت	همی آید از هر سوی تیغ تفت"
print(f"text: {text}")

encoded = tk.encode(text)
print(f"\ntokens: {encoded.tokens}")
print(f"\ntoken ids: {encoded.ids}")

decoded = tk.decode(encoded.ids)
print(f"\ndecoded text: {decoded}")




text:  چو ترکان بدیدند کارجاسپ رفت	همی آید از هر سوی تیغ تفت

tokens: [' ', 'چو ', 'ترکان ب', 'دیدند ', 'کارج', 'اسپ ', 'رفت', '\tهمی ', 'آید از هر سوی ', 'تیغ ', 'تفت']

token ids: [7, 111, 2951, 2165, 29842, 515, 157, 294, 15461, 716, 27223]

decoded text:   چو  ترکان ب دیدند  کارج اسپ  رفت 	همی  آید از هر سوی  تیغ  تفت


In [173]:
encoded = tk.encode_batch([text])
[enc.ids for enc in encoded]

[[7, 111, 2951, 2165, 29842, 515, 157, 294, 15461, 716, 27223]]

In [174]:
tk.get_vocab_size()

30000

In [176]:
token = "تیغ"
id = tk.token_to_id(token)
print(f"id: {id}, token: {tk.id_to_token(id)}")
print([tk.id_to_token(i) for i in range(20)])

id: 1105, token: تیغ
['[UNK]', '[CLS]', '[SEP]', '[PAD]', '[MASK]', '\t', '\n', ' ', '(', ')', '«', '»', '،', '؟', 'ء', 'آ', 'أ', 'ؤ', 'ئ', 'ا']


In [8]:
weights = torch.randn(5, 5)
m, n = weights.shape
weights

tensor([[ 0.4460, -1.1201, -0.9348, -1.4863,  0.9522],
        [-0.4075, -1.2803,  0.8713, -3.7395,  1.7959],
        [ 1.7201, -0.3779, -0.7992, -2.0305, -0.4960],
        [ 1.5586,  1.2785, -0.2851,  0.2826,  0.4734],
        [-1.0473, -1.5379, -2.1796,  1.1212,  0.3986]])

In [12]:
weights.masked_fill(torch.tril(torch.ones(m, n)) == 0, float('-inf'))

tensor([[ 0.4460,    -inf,    -inf,    -inf,    -inf],
        [-0.4075, -1.2803,    -inf,    -inf,    -inf],
        [ 1.7201, -0.3779, -0.7992,    -inf,    -inf],
        [ 1.5586,  1.2785, -0.2851,  0.2826,    -inf],
        [-1.0473, -1.5379, -2.1796,  1.1212,  0.3986]])

In [179]:
def split(data, train_ratio=0.8, val_ratio=0.1):
    src, tgt = data
    assert len(src) == len(tgt), "expeted the same source and target sizes."

    ntrain = int(train_ratio * len(src))
    nval = int(val_ratio * len(src))
    indices = torch.randperm(len(src))

    train = (src[indices][:ntrain], tgt[indices][:ntrain])
    val = (src[indices][ntrain : ntrain + nval], tgt[indices][ntrain : ntrain + nval])
    test = (src[indices][ntrain + nval :], tgt[indices][ntrain + nval :])

    return train, val, test

def get_batch(data, batch_size, device):
    """
    Generates a batch of examples.
    """
    src, tgt = data
    assert len(src) == len(tgt), "expeted the same source and target sizes."

    indices = torch.randint(len(src), (batch_size,))

    x = src[indices].to(device)
    y = tgt[indices].to(device)

    return x, y

In [201]:
with open("data/shahname_src.txt") as f:
    src = f.read().splitlines()

src = torch.tensor([x.ids for x in tk.encode_batch(src)], dtype=torch.long)
print(f"source batch data size: {src.shape}")

with open("data/shahname_tgt.txt") as f:
    tgt = f.read().splitlines(keepends=True)

tgt = torch.tensor([x.ids for x in tk.encode_batch(tgt)], dtype=torch.long)
print(f"target batch data size: {tgt.shape}")

print("'" + tk.decode(tgt[:1][0].tolist()) + "'")
print(tgt[:1][0])
print([tk.id_to_token(id) for id in tgt[:1][0]])

train, val, _ = split((src, tgt), 0.9, 0.1)


source batch data size: torch.Size([49609, 13])
target batch data size: torch.Size([49609, 10])
'	همی  آید از هر سوی  تیغ  تفت
'
tensor([  294, 15461,   716,  1741,     3,     3,     3,     3,     3,     3])
['\tهمی ', 'آید از هر سوی ', 'تیغ ', 'تفت\n', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


In [181]:
src, tgt = get_batch(train, 2, "cpu")
src, tgt

(tensor([[  571, 26770,  1600,  4257,  1019,     3,     3,     3,     3,     3,
              3,     3,     3],
         [10837, 11853,  1109,   521,     3,     3,     3,     3,     3,     3,
              3,     3,     3]]),
 tensor([[2087, 1985,  547,  693, 1106,    3,    3,    3,    3,    3],
         [   5,  549,  334, 5997, 5672, 1093,  554,    3,    3,    3]]))

In [168]:
with open("data/shahname.txt") as f:
    lines = f.read().splitlines(keepends=True)

lines = lines[:10]

lines = [line.split("\t") for line in lines]
src = [line[0] for line in lines if len(line) == 2]
tgt = ["\t"+line[1] for line in lines if len(line) == 2]
src[:5], tgt[:5]


(['چو ترکان بدیدند کارجاسپ رفت',
  'همه سرکشانشان پیاده شدند',
  'کمانچای چاچی بینداختند',
  'به زاریش گفتند گر شهریار',
  'بدین اندر آییم و خواهش کنیم'],
 ['\tهمی آید از هر سوی تیغ تفت\n',
  '\tبه پیش گو اسفندیار آمدند\n',
  '\tقبای نبردی برون آختند\n',
  '\tدهد بندگان را به جان زینهار\n',
  '\tهمه آذران را نیایش کنیم\n'])

In [162]:
with open("data/shahname.txt") as f:
    lines = f.read().splitlines(keepends=True)

src_token_ids = torch.tensor([x.ids for x in tk.encode_batch(src_lines)], dtype=torch.long)

['چو ترکان بدیدند کارجاسپ رفت\tهمی آید از هر سوی تیغ تفت\n',
 'همه سرکشانشان پیاده شدند\tبه پیش گو اسفندیار آمدند\n',
 'کمانچای چاچی بینداختند\tقبای نبردی برون آختند\n',
 'به زاریش گفتند گر شهریار\tدهد بندگان را به جان زینهار\n',
 'بدین اندر آییم و خواهش کنیم\tهمه آذران را نیایش کنیم\n',
 'ازیشان چو بشنید اسفندیار\tبه جان و به تن دادشان زینهار\n',
 'بران لشگر گشن آواز داد\tگو نامبردار فرخ نژاد\n',
 'که این نامداران ایرانیان\tبگردید زین لشکر چینیان\n',
 'کنون کاین سپاه عدو گشت پست\tازین سهم و کشتن بدارید دست\n',
 'که بس زاروارند و بیچاره وار\tدهدی این سگان را به جان زینهار\n']

In [205]:
first_mesra = "چو ترکان بدیدند کارجاسپ رفت"
tk.encode(first_mesra).ids

[111, 2951, 2165, 29842, 515, 157]

In [212]:
a = torch.ones(2,2)
a = torch.cat([torch.zeros(a.shape[0], 1), a], dim=1)
a

tensor([[0., 1., 1.],
        [0., 1., 1.]])

In [225]:
def get_tokenizer(filenames):
    tk = Tokenizer(BPE(unk_token="[UNK]"))
    tk.enable_padding(pad_id=3)

    trainer = BpeTrainer(special_tokens=["[SOS]", "[EOS]", "[UNK]", "[PAD]"])
    tk.train(filenames, trainer)

    return tk

text_file = ["data/hafez.txt"]
tk = get_tokenizer(text_file)

Exception: stream did not contain valid UTF-8

In [243]:

with open(text_file[0]) as f:
    text = f.read()

token_ids = torch.tensor(tk.encode(text).ids, dtype=torch.long)

token_ids[:10]

tensor([21808,   304,     7,     7,     0,     6,     6,  1748,     0,    71])